# Appendix 10.1: Chaining Prompts

（付録 10.1：プロンプトの連鎖（チェイニング））

- [レッスン](#lesson)
- [例のプレイグラウンド](#example-playground)

## Setup

次のセットアップセルを実行して、APIキーの読み込みと `get_completion` ヘルパー関数の準備を行います。

In [13]:
#%pip install anthropic --quiet

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION
MODEL_NAME = "us.anthropic.claude-3-haiku-20240307-v1:0"

client = AnthropicBedrock(aws_region=AWS_REGION)

# def get_completion(messages, system_prompt=''):
#     message = client.messages.create(
#         model=MODEL_NAME,
#         max_tokens=2000,
#         temperature=0.0,
#         messages=messages,
#         system=system_prompt
#     )
#     return message.content[0].text
def get_completion(messages, system_prompt=''):
    message = ''
    stream = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages,
        system=system_prompt,
        stream=True
    )
    for event in stream:
        if event.type == "content_block_delta":
            print(event.delta.text, flush=True, end="")
            message += event.delta.text
    return message

---

## Lesson

「文章は推敲してこそ」という言葉があります。実は **Claude も、そう頼まれると応答の精度を改善できることがよくあります**！

Claude に「もう一度考えて」と促す方法はたくさんあります。人間に『確認して』『見直して』と頼むのが自然に感じられる言い方は、一般に Claude に対しても有効です。（プロンプトチェイニングをいつ・どのように使うかの例は、[prompt chaining documentation](https://docs.anthropic.com/claude/docs/chain-prompts) も参照してください。）

### Examples

この例では、Claude に10個の単語を挙げさせます……が、そのうち1つ以上が実在しない単語になってしまいます。

In [14]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
# print(first_response)

Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Fab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Zab

**『もっと正確にして』と Claude に頼む** だけで、このエラーは修正できます！

下では、上の Claude の誤った応答を `first_response` に入れ、会話ターンを1つ追加して『さっきの回答を直して』と依頼しています。

In [15]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
# print(get_completion(messages))
message=get_completion(messages)

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Fab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Zab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words."}]

------------------------------------- Claude's response -------------------------------------
Here are 10 real words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab

でも、Claude は『直して』と言われたから機械的に直しているだけなのでしょうか？ では最初から正しい答えを与えたらどうなるでしょう？ Claude は自信を失ってしまうのでしょうか？ ここでは `first_response` に正しい応答を入れて、もう一度ダブルチェックさせています。

In [20]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
message=get_completion(messages)

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words."}]

------------------------------------- Claude's response -------------------------------------
Here are 10 real words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab

上のブロックを何度か実行してみると分かりますが、多くの場合 Claude は単語をそのまま維持する一方で、すべて正しいにもかかわらず、ときどき単語を変えてしまうことがあります。これを緩和するにはどうすればよいでしょう？ 第8章と同様に、Claude に“逃げ道（out）”を与えられます。もう一度試してみましょう。

In [18]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
message=get_completion(messages)

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."}]

------------------------------------- Claude's response -------------------------------------
Okay, here is the list with replacements for any words that are not real:

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab

All of these are real words, so the original list is returned.

上のコードを何度か実行してみてください。今度は Claude が自分の答えを維持（stick to its guns）しやすくなっているはずです。

プロンプトチェイニングは、**Claude に『もっと良くして』と頼む** 目的にも使えます。下では、まず Claude に物語を書かせ、その後で自分が書いた物語を改善させています。好みは分かれるかもしれませんが、多くの人は2版目の方が良いと感じるでしょう。

まずは、Claude の初稿を生成します。

In [21]:
# Initial prompt
# first_user = "Write a three-sentence short story about a girl who likes to run."
first_user = "走るのが好きな女の子についての 3 文の短編小説を書いてください。"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
# print(first_response)

はい、ご要望の通り、走るのが好きな女の子についての3文の短編小説を書きました。

走るのが好きな女の子

朝日が差し込む中、彼女は公園を駆け抜けていく。軽やかな足取りで、風に吹かれながら自由に走る。

走ることで心が晴れ渡り、心地よい疲労感が体を包む。走り終えた後の充実感は格別だ。

彼女にとって、走ることは単なる運動以上の意味を持っている。走ることが、自分らしさを表現する方法なのだ。

次に、Claude に初稿を改善させてみましょう。

In [22]:
# second_user = "Make the story better."
second_user = "物語を改善してください。"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
message=get_completion(messages)

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': '走るのが好きな女の子についての 3 文の短編小説を書いてください。'}, {'role': 'assistant', 'content': 'はい、ご要望の通り、走るのが好きな女の子についての3文の短編小説を書きました。\n\n走るのが好きな女の子\n\n朝日が差し込む中、彼女は公園を駆け抜けていく。軽やかな足取りで、風に吹かれながら自由に走る。\n\n走ることで心が晴れ渡り、心地よい疲労感が体を包む。走り終えた後の充実感は格別だ。\n\n彼女にとって、走ることは単なる運動以上の意味を持っている。走ることが、自分らしさを表現する方法なのだ。'}, {'role': 'user', 'content': '物語を改善してください。'}]

------------------------------------- Claude's response -------------------------------------
はい、ご要望に応じて物語を改善しました。より詳細で生き生きとした描写を心がけました。

走るのが好きな女の子

朝日が差し込む中、彼女は公園の小道を軽やかに駆け抜けていく。柔らかな風が髪を揺らめかせ、心地よい疲労感が体を包む。

走ることで心が晴れ渡り、自由に動き回れる喜びが湧き上がってくる。走り終えた後の充実感は格別で、心が洗われたような爽快感に包まれる。

彼女にとって、走ることは単なる運動以上の意味を持っている。走ることが、自分らしさを表現し、心の奥底にある何かを解放する方法なのだ。走ることで、彼女は自分の存在を感じ、生きる喜びを見出すのだ。

この種の置換（substitution）はとても強力です。これまで、置換プレースホルダーを使ってリストや単語、Claude の過去の応答などを差し込んできました。同様に、置換を使って **いわゆる「function calling（関数呼び出し）」** も行えます。これは、Claude に何らかの関数を呼ばせ、その関数の結果を受け取って、さらにその結果を使って次のタスクをさせる、という流れです。仕組みとしては他の置換と同じです。詳しくは次の付録で扱います。

下では、Claude を1回呼び出した結果を、別の（より長い）呼び出しに差し込む例をもう一つ示します。まず最初のプロンプトから始めましょう（今回は Claude の応答の prefill も含みます）。

In [23]:
# first_user = """Find all names from the below text:

# "Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""
first_user = """以下のテキストからすべての名前を見つけてください。

「こんにちは、ジェシー。私、エリンです。ジョーイが明日開くパーティーについて電話しました。ケイシャが来ると言っていましたし、メルも来ると思います。」"""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
# print(first_response)


- ジェシー
- エリン
- ジョーイ
- ケイシャ
- メル
</names>------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': '以下のテキストからすべての名前を見つけてください。\n\n「こんにちは、ジェシー。私、エリンです。ジョーイが明日開くパーティーについて電話しました。ケイシャが来ると言っていましたし、メルも来ると思います。」'}, {'role': 'assistant', 'content': '<names>'}]

------------------------------------- Claude's response -------------------------------------


この名前リストを、別のプロンプトに渡してみましょう。

In [24]:
# second_user = "Alphabetize the list."
second_user = "リストをアルファベット順に並べ替えてください。"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
message=get_completion(messages)

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': '以下のテキストからすべての名前を見つけてください。\n\n「こんにちは、ジェシー。私、エリンです。ジョーイが明日開くパーティーについて電話しました。ケイシャが来ると言っていましたし、メルも来ると思います。」'}, {'role': 'assistant', 'content': '<names>\n\n- ジェシー\n- エリン\n- ジョーイ\n- ケイシャ\n- メル\n</names>'}, {'role': 'user', 'content': 'リストをアルファベット順に並べ替えてください。'}]

------------------------------------- Claude's response -------------------------------------
<names>
- エリン
- ジェシー
- ジョーイ
- ケイシャ
- メル
</names>

プロンプトチェイニングを学んだところで、付録 10.2 に進み、プロンプトチェイニングを使って function calling（ツール利用）を実装する方法を学びましょう。

---

## Example Playground

このエリアでは、このレッスンで示したプロンプト例を自由に試し、プロンプトを調整して Claude の応答がどう変わるかを観察できます。

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))